In [1]:
import emstereolibpy as em
import pandas as pd
import csv

from emstereolibpy import CameraID

Check that we're up and running

In [2]:
em.version()

(1, 0)

Pick the file we wish to transform

In [3]:
from tkinter import Tk
from tkinter import filedialog

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = filedialog.askopenfilename()

In [5]:
df_track = pd.read_csv(filename)
df_track.head()

,frame_left,id_left,color_left,ltrb_left,ltwh_left,center_left,frame_right,id_right,color_right,ltrb_right,ltwh_right,center_right,FilenameLeft,FilenameRight,CamCalFile,Out_FileName
0,73884,10,"(255, 152, 0)","(95.0, 98.0, 443.0, 376.0)","(95.0, 98.0, 348.0, 278.0)","(269.0, 237.0)",74204,13,"(4, 204, 0)","(254.0, 102.0, 754.0, 344.0)","(254.0, 102.0, 500.0, 242.0)","(504.0, 223.0)",A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi
1,73885,10,"(255, 152, 0)","(92.0, 87.0, 461.0, 385.0)","(92.0, 87.0, 369.0, 298.0)","(276.5, 236.0)",74205,13,"(4, 204, 0)","(265.0, 103.0, 768.0, 344.0)","(265.0, 103.0, 503.0, 241.0)","(516.5, 223.5)",A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi
2,73886,10,"(255, 152, 0)","(128.0, 78.0, 485.0, 369.0)","(128.0, 78.0, 357.0, 291.0)","(306.5, 223.5)",74206,13,"(4, 204, 0)","(277.0, 101.0, 783.0, 335.0)","(277.0, 101.0, 506.0, 234.0)","(530.0, 218.0)",A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi
3,73887,10,"(255, 152, 0)","(131.0, 83.0, 492.0, 350.0)","(131.0, 83.0, 361.0, 267.0)","(311.5, 216.5)",74207,13,"(4, 204, 0)","(295.0, 98.0, 788.0, 335.0)","(295.0, 98.0, 493.0, 237.0)","(541.5, 216.5)",A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi
4,73888,10,"(255, 152, 0)","(133.0, 82.0, 501.0, 341.0)","(133.0, 82.0, 368.0, 259.0)","(317.0, 211.5)",74208,13,"(4, 204, 0)","(292.0, 101.0, 795.0, 325.0)","(292.0, 101.0, 503.0, 224.0)","(543.5, 213.0)",A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi


In [6]:
em.load_camera_file(CameraID.left, 'data/6429_RIG1_L.Cam')
em.load_camera_file(CameraID.right, 'data/6429_RIG1_R.Cam')

0

In [31]:
glob_tl_coords = []
glob_br_coords = []
x1 = 0
y1 = 1
x2 = 2
y2 = 3

glob_coords_dict = {
    'tlrb_xyz': [],
    'center_xyz': [],
    'tlrb_sd_xyz': [],
    'tlrb_rms': [],
    'center_sd_xyz': [],
    'center_rms': []
}

for ii, row in df_track.iterrows():

    # <camera>_<pixel>
    left_coord = tuple(float(num) for num in df_track.ltrb_left[ii].replace('(', '').replace(')', '').split(', '))  # convert string  '(x1, y1, x2, y2)' to tuple of floats.
    right_coord = tuple(float(num) for num in df_track.ltrb_right[ii].replace('(', '').replace(')', '').split(', '))
    centre_left_coord = tuple(float(num) for num in df_track.center_left[ii].replace('(', '').replace(')', '').split(', '))
    centre_right_coord = tuple(float(num) for num in df_track.center_right[ii].replace('(', '').replace(')', '').split(', '))

    pt_left = em.Pt2D(left_coord[x1], left_coord[y1])   #top left of left BB
    pt_right = em.Pt2D(right_coord[x1], right_coord[y1]) # top left of right BB

    glob_tl_coords, glob_tl_sd, glob_tl_rms = em.intersect(pt_left, pt_right)

    pt_left = em.Pt2D(left_coord[x2], left_coord[y2])   #bottom right of left BB
    pt_right = em.Pt2D(right_coord[x2], right_coord[y2]) #Bottom right of right BB

    glob_br_coords, glob_br_sd, glob_br_rms = em.intersect(pt_left, pt_right)

    centre_pt_left = em.Pt2D(centre_left_coord[0], centre_left_coord[1])
    centre_pt_right = em.Pt2D(centre_right_coord[0], centre_right_coord[1])
    glob_centre_pt, glob_centre_sd, glob_centre_rms = em.intersect(centre_pt_left, centre_pt_right)

    glob_coords_dict['tlrb_xyz'].append(glob_tl_coords + glob_br_coords)
    glob_coords_dict['tlrb_sd_xyz'].append(glob_tl_sd + glob_br_sd)
    glob_coords_dict['tlrb_rms'].append((glob_tl_rms, glob_br_rms))

    glob_coords_dict['center_xyz'].append(glob_centre_pt)
    glob_coords_dict['center_sd_xyz'].append(glob_tl_sd + glob_br_sd)
    glob_coords_dict['center_rms'].append((glob_centre_rms, glob_centre_rms))


<class 'int'>


In [33]:
df_track['tlrb_xyz'] = glob_coords_dict['tlrb_xyz']
df_track['tlrb_sd_xyz'] = glob_coords_dict['tlrb_sd_xyz']
df_track['tlrb_rms'] = glob_coords_dict['tlrb_rms']

df_track['center_xyz'] = glob_coords_dict['center_xyz']
df_track['center_sd_xyz'] = glob_coords_dict['center_sd_xyz']
df_track['center_rms'] = glob_coords_dict['center_rms']

In [34]:
df_track.head()

,frame_left,id_left,color_left,ltrb_left,ltwh_left,center_left,frame_right,id_right,color_right,ltrb_right,...,FilenameLeft,FilenameRight,CamCalFile,Out_FileName,tlrb_xyz,center_xyz,tlrb_sd_xyz,tlrb_rms,center_sd_xyz,center_rms
0,73884,10,"(255, 152, 0)","(95.0, 98.0, 443.0, 376.0)","(95.0, 98.0, 348.0, 278.0)","(269.0, 237.0)",74204,13,"(4, 204, 0)","(254.0, 102.0, 754.0, 344.0)",...,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,"(-699.9707405795988, 419.67461590045997, -1619...","(-610.268790528276, 367.82899795038617, -1944....","(1.2315116196547633, 0.9609239291922759, 2.900...","(18.041096166506275, 16.9389504141414)","(1.2315116196547633, 0.9609239291922759, 2.900...","(15.573309260244729, 15.573309260244729)"
1,73885,10,"(255, 152, 0)","(92.0, 87.0, 461.0, 385.0)","(92.0, 87.0, 369.0, 298.0)","(276.5, 236.0)",74205,13,"(4, 204, 0)","(265.0, 103.0, 768.0, 344.0)",...,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,"(-708.9594468258505, 432.4102255603853, -1650....","(-606.155247477723, 372.1769556959787, -1966.3...","(1.2701737741541732, 0.9969224332071057, 3.009...","(13.365973495932383, 22.102126649189486)","(1.2701737741541732, 0.9969224332071057, 3.009...","(14.776430587291014, 14.776430587291014)"
2,73886,10,"(255, 152, 0)","(128.0, 78.0, 485.0, 369.0)","(128.0, 78.0, 357.0, 291.0)","(306.5, 223.5)",74206,13,"(4, 204, 0)","(277.0, 101.0, 783.0, 335.0)",...,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,"(-674.0552726716334, 428.21614877790165, -1615...","(-574.7800817414699, 375.70903976269807, -1934...","(1.1861800399238756, 0.9662349594555513, 2.875...","(10.132065274114582, 18.032180330017937)","(1.1861800399238756, 0.9662349594555513, 2.875...","(11.177607516868068, 11.177607516868068)"
3,73887,10,"(255, 152, 0)","(131.0, 83.0, 492.0, 350.0)","(131.0, 83.0, 361.0, 267.0)","(311.5, 216.5)",74207,13,"(4, 204, 0)","(295.0, 98.0, 788.0, 335.0)",...,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,"(-678.9563234773814, 436.7265232997833, -1653....","(-572.8099869198903, 384.84526137225816, -1957...","(1.2201342570161149, 0.998474591355122, 3.0024...","(13.421786902343904, 6.738301883143432)","(1.2201342570161149, 0.998474591355122, 3.0024...","(8.588234129050129, 8.588234129050129)"
4,73888,10,"(255, 152, 0)","(133.0, 82.0, 501.0, 341.0)","(133.0, 82.0, 368.0, 259.0)","(317.0, 211.5)",74208,13,"(4, 204, 0)","(292.0, 101.0, 795.0, 325.0)",...,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,"(-675.3089044695449, 433.16532344828295, -1642...","(-566.8644627484978, 387.804890038285, -1950.3...","(1.206309365946083, 0.9870193373391696, 2.9639...","(11.592542215520115, 7.717495940883574)","(1.206309365946083, 0.9870193373391696, 2.9639...","(7.98924204575693, 7.98924204575693)"


In [35]:
df_track.to_csv(filename.strip('.csv')+'-global.csv')